In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-121655")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-121655
Azure region: southcentralus
Subscription id: 8e713106-916f-4177-890e-435b90d7adc4
Resource group: aml-quickstarts-121655


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

from azureml.core.compute_target import ComputeTargetException


cpu_cluster_name = "BankCC "

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [2]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.core import Dataset
import os

# Specify parameter sampler
ps = RandomParameterSampling(
	{
		'--C':uniform(.90, 1.10),
		'--max_iter':uniform(90, 100)
	}
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./', compute_target=compute_target, entry_script='train.py', pip_packages=[])

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est, hyperparameter_sampling=ps, policy=policy, primary_metric_name='Accuracy', primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, max_total_runs=20, max_concurrent_runs=4)

TrainingException: TrainingException:
	Message: SKLearn 2.0 is not currently supported by the SKLearn estimator. Check https://docs.microsoft.com/python/api/azureml-train-core/azureml.train.dnn?view=azure-ml-py for all supported versions. To use SKLearn 2.0, switch to the generic Estimator class for your experiment.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "SKLearn 2.0 is not currently supported by the SKLearn estimator. Check https://docs.microsoft.com/python/api/azureml-train-core/azureml.train.dnn?view=azure-ml-py for all supported versions. To use SKLearn 2.0, switch to the generic Estimator class for your experiment."
    }
}

In [8]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hdr = exp.submit(config=hyperdrive_config)
RunDetails(hdr).show()

NameError: name 'hyperdrive_config' is not defined

In [3]:
import joblib
# Get your best run and save the model from that run.

joblib.dump(value = reg, filename=model_name)

In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
file_url = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
dsauto = Dataset.Tabular.from_delimited_files(path = file_url)

In [3]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(dsauto)

df = ds.to_pandas_dataframe()
df.head()df = ds.to_pandas_dataframe()
df.head()

NameError: name 'Dataset' is not defined

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="Marketing",
    primary_metric="Accuracy",
    training_data="x_train",
    label_column_name="y",
    n_cross_validations=5)

In [2]:
# Submit your automl run
from azureml.train.automl import Experiment
experiment = Experiment(ws, "automl_test_experiment")
run = experiment.submit(config =automl_config, show_output = True)

In [4]:
# Retrieve and save your best automl model.
import joblib
joblib.dump(value = reg, filename=model_name)

NameError: name 'reg' is not defined